In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore")

In [2]:
file = pd.ExcelFile(r"C:\Users\OrganDonor_Consolidated.xlsx")
donor = pd.read_excel(file,'DonorDetails')
referral = pd.read_excel(file,"ReferralDetails")
outcomes = pd.read_excel(file,"Outcomes")
opo_hosp = pd.read_excel(file,'OPO_HospDetails')
yearly = pd.read_excel(file,"Yearly_Outcomes")

# 1. which mechanism of death or cause of death causes more brain deaths

In [3]:
filtered_donor = donor[donor['brain_death'] == 1]

# Group by mechanism_of_death and count occurrences
mechanism_counts = (
    filtered_donor.groupby('mechanism_of_death')
    .size()
    .reset_index(name='Brain Death Count')
    .sort_values(by='Brain Death Count', ascending=False)
)

# Group by causeofdeath and count occurrences
cause_counts = (
    filtered_donor.groupby('causeofdeath')
    .size()
    .reset_index(name='Brain Death Count')
    .sort_values(by='Brain Death Count', ascending=False)
)

# Display results
print("Mechanism of Death with Most Brain Deaths:")
print(mechanism_counts)

print("\nCause of Death with Most Brain Deaths:")
print(cause_counts)

# Compare the highest counts from both groups
highest_mechanism = mechanism_counts.iloc[0]
highest_cause = cause_counts.iloc[0]

print("\nComparison:")
if highest_mechanism['Brain Death Count'] > highest_cause['Brain Death Count']:
    print(
        f"The mechanism of death '{highest_mechanism['mechanism_of_death']}' "
        f"caused the most brain deaths with a count of {highest_mechanism['Brain Death Count']}."
    )
elif highest_cause['Brain Death Count'] > highest_mechanism['Brain Death Count']:
    print(
        f"The cause of death '{highest_cause['causeofdeath']}' "
        f"caused the most brain deaths with a count of {highest_cause['Brain Death Count']}."
    )
else:
    print(
        f"Both the mechanism of death '{highest_mechanism['mechanism_of_death']}' "
        f"and the cause of death '{highest_cause['causeofdeath']}' "
        f"have the same highest brain death count of {highest_mechanism['Brain Death Count']}."
    )



Mechanism of Death with Most Brain Deaths:
           mechanism_of_death  Brain Death Count
15                     Stroke               3694
2              Cardiovascular               2266
1                Blunt Injury               2052
5           Drug Intoxication               1129
7              Gun Shot Wound                730
0                Asphyxiation                554
9              Natural Causes                458
10          None of the Above                350
8               Gunshot Wound                170
4                    Drowning                140
17                    Unknown                110
13                    Seizure                109
3   Death from Natural Causes                109
14                       Stab                 22
6                  Electrical                 12
12                      Other                 12
16        Sudden Infant Death                  9
11          None of the above                  5

Cause of Death with Most 

# 2. which hospital recorded max num of brain deaths

In [4]:
# Merge Donor and Hospital sheet
merged_data = pd.merge(donor, opo_hosp, on="PatientID", how="inner")

# Get the list of Brain dead 
filtered_donor = merged_data[merged_data['brain_death'] == 1]

# Group by hospital id to get based on hospitals
Brain_Death_counts = filtered_donor.groupby('HospitalID').size().reset_index(name='BrainDeathCount')

#sort the count in descending order to show all the list of hospitals having brain dead
Brain_Death_counts_sorted = Brain_Death_counts.sort_values(by='BrainDeathCount', ascending=False)

# Print the sorted result
print("List of Hospitals sorted based on number of Brain Deaths:")
print(Brain_Death_counts_sorted)

#Getting the Hospital wih Maximum count of Brain death
max_count = Brain_Death_counts['BrainDeathCount'].max()

# Filter rows with the maximum count
hospitals_with_max_count = Brain_Death_counts[Brain_Death_counts['BrainDeathCount'] == max_count]

# Display the result.... The Answer shows all hospitals with Maximum number of Brain Deaths
print("Hospital(s) with the Maximum number of Brain Deaths:")
print(hospitals_with_max_count)

List of Hospitals sorted based on number of Brain Deaths:
      HospitalID  BrainDeathCount
334  OPO6_H22175              390
270  OPO5_H10831              345
52   OPO1_H27265              327
291   OPO5_H1925              315
154   OPO3_H4291              297
..           ...              ...
21   OPO1_H20007                1
167  OPO4_H10855                1
20   OPO1_H19921                1
162   OPO3_H9271                1
298   OPO5_H4502                1

[350 rows x 2 columns]
Hospital(s) with the Maximum number of Brain Deaths:
      HospitalID  BrainDeathCount
334  OPO6_H22175              390


# 3. Max number of males donated which organ and same with females

In [5]:
#Merging the donor and outcomes sheet
merged_don_out = pd.merge(donor, outcomes, on="PatientID", how="inner")

# Getting the list of organs as columns
organ_columns = [col for col in merged_don_out.columns if col.startswith('outcome_')]

# Create a new DataFrame to store results for each organ type
organ_gender_counts = []

# Iterate through each organ column
for organ in organ_columns:
    # Filter rows where the transplant occurred for this organ
    organ_transplants = merged_don_out[merged_don_out[organ] == 'Transplanted']
    
    # Group by gender and count occurrences
    organ_gender_count = organ_transplants.groupby('gender').size().reset_index(name='Count')
    
    # Add the organ type column
    organ_gender_count['Organ'] = organ.replace('outcome_', '')  # Strip 'outcome_' from the column name
    
    # Append to the list
    organ_gender_counts.append(organ_gender_count)

# Concatenate all results into a single DataFrame
organ_gender_counts_df1 = pd.concat(organ_gender_counts, ignore_index=True)

# Sort by 'Count' in descending order
organ_gender_counts_df1 = organ_gender_counts_df1.sort_values(by='Count', ascending=False)

# Separate the results for each gender
male_results = organ_gender_counts_df1[organ_gender_counts_df1['gender'] == 'M']
female_results = organ_gender_counts_df1[organ_gender_counts_df1['gender'] == 'F']

# Display the result for Male
print("Male Results:")
print(male_results)

# Display the result for Female
print("Female Results:")
print(female_results)

#Getting the Gender Maximum count of organs transplanted
male_max_count = male_results['Count'].max()
female_max_count = female_results['Count'].max()

# Filter rows with the maximum count
Organ_Gender_with_male_max_count = organ_gender_counts_df1[organ_gender_counts_df1['Count'] == male_max_count]
Organ_Gender_with_female_max_count = organ_gender_counts_df1[organ_gender_counts_df1['Count'] == female_max_count]

# Display the result.... 
print("Max number of males donated organ:")
print(Organ_Gender_with_male_max_count )

print("Max number of females donated organ:")
print(Organ_Gender_with_female_max_count )

Male Results:
   gender  Count         Organ
5       M   4494   kidney_left
7       M   4467  kidney_right
3       M   4316         liver
1       M   2106         heart
9       M   1284     lung_left
11      M   1274    lung_right
15      M    642      pancreas
13      M     49     intestine
Female Results:
   gender  Count         Organ
2       F   2698         liver
4       F   2590   kidney_left
6       F   2585  kidney_right
0       F    881         heart
8       F    862     lung_left
10      F    853    lung_right
14      F    250      pancreas
12      F     36     intestine
Max number of males donated organ:
  gender  Count        Organ
5      M   4494  kidney_left
Max number of females donated organ:
  gender  Count  Organ
2      F   2698  liver


# 4. Create a Python function that filters the records where the referral_time falls within a specific date range (e.g., between October 2028 to December 2028).

In [8]:
def filter_by_date_range(referral, starting, ending, date_column='time_referred'):

    # Filter rows where the date is within the specified range
    filtered_df1 = referral[(referral[date_column] >= pd.to_datetime(starting)) & (referral[date_column] <= pd.to_datetime(ending))]

    return filtered_df1

# Define date range for filtering
starting = '2028-10-01'
ending = '2028-12-31'

# Call the function
filtered_donors = filter_by_date_range(referral, starting, ending)

# Display the filtered results
print(filtered_donors)

           PatientID  Approached Relatives  Authorized By Family  \
1206    OPO1_P427408                     1                     1   
1268    OPO1_P466695                     0                     0   
1282    OPO1_P443842                     1                     1   
3017    OPO1_P592531                     0                     0   
3652    OPO1_P180417                     0                     0   
...              ...                   ...                   ...   
131325  OPO6_P273109                     0                     0   
131418  OPO6_P974781                     0                     0   
131698  OPO6_P397649                     0                     0   
132649  OPO6_P863421                     0                     0   
133021  OPO6_P434091                     0                     0   

        Cross_Clamped  transplanted  Tissue_Referral  Eye_Referral  \
1206                1             1                1             1   
1268                0             0        

# 5. Write a Python script that identifies and displays any duplicate donor records, and then creates a new DataFrame that includes only the most recent referral.


In [9]:
def filter_duplicates_and_keep_most_recent(df, donor_id_column='PatientID', date_column='time_referred'):
   
    # Sort the DataFrame by donor_id_column and date_column (descending to get most recent referrals first)
    df_sorted = referral.sort_values(by=[donor_id_column, date_column], ascending=[True, False])

    # Identify duplicate rows based on the donor_id_column (keep the first occurrence, which is the most recent due to sorting)
    df_no_duplicates = df_sorted.drop_duplicates(subset=[donor_id_column], keep='first')

    # Display duplicate records (if any)
    duplicates = referral[referral.duplicated(subset=[donor_id_column], keep=False)]
    if not duplicates.empty:
        print("Duplicate Donor Records Found:")
        print(duplicates)
    else:
        print("No duplicate donor records found.")

    # Return the DataFrame with the most recent referral for each donor
    return df_no_duplicates

# Call the function
most_recent_donors = filter_duplicates_and_keep_most_recent(referral, donor_id_column='PatientID', date_column='time_referred')

# Display only selected columns from the most recent donor records
selected_columns = ['PatientID', 'time_referred']  # List the columns you want to display

# Display the result with selected columns
print("Most Recent Donor Records (Selected Columns):")
print(most_recent_donors[selected_columns])

No duplicate donor records found.
Most Recent Donor Records (Selected Columns):
           PatientID           time_referred
31900     OPO1_P1000 2033-06-21 22:45:30.263
17805   OPO1_P100009 2034-08-11 15:01:25.340
24490    OPO1_P10004 2036-04-13 00:02:01.093
6171    OPO1_P100071 2036-05-10 15:33:33.273
414     OPO1_P100077 2035-09-05 12:33:33.140
...              ...                     ...
127847  OPO6_P999867 2031-12-30 19:50:00.000
114449  OPO6_P999884 2032-01-05 12:52:00.000
118306  OPO6_P999899 2034-07-01 05:07:00.000
115631  OPO6_P999923 2031-09-20 09:06:00.000
111268   OPO6_P99994 2030-08-23 05:14:00.000

[133101 rows x 2 columns]


# 6. Provide the number of donors by each day

In [11]:
# Group by 'Referral_DayofWeek' and count the occurrences of 'PatientID'
day_of_week_patient_count = referral.groupby('Referral_DayofWeek')['PatientID'].count().reset_index(name='Patient Count')

# Sort the days of the week in the correct order (Monday to Sunday)
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_of_week_patient_count = day_of_week_patient_count.set_index('Referral_DayofWeek').reindex(days_order).reset_index()

# Rename columns for clarity
day_of_week_patient_count.columns = ['Day of Week', 'Patient Count']

# Display the result
print("Donor Count by Day of the Week:")
print(day_of_week_patient_count)

Donor Count by Day of the Week:
  Day of Week  Patient Count
0      Monday          19451
1     Tuesday          19623
2   Wednesday          19421
3    Thursday          18900
4      Friday          19346
5    Saturday          18077
6      Sunday          18283


# 7. What is the ratio of Eye donations to Tissue donations?

In [12]:
# Count the number of rows where Eye_Referral and Tissue_Referral are 1
donor_eye_count = referral[referral['Eye_Referral'] == 1].shape[0]
donor_tissue_count = referral[referral['Tissue_Referral'] == 1].shape[0]

if donor_tissue_count != 0:  # Prevent division by zero
    Tissue_eye_Ratio = donor_eye_count / donor_tissue_count
    if not np.isnan(Tissue_eye_Ratio):
        Tissue_eye_Ratio = round(Tissue_eye_Ratio, 1)
    else:
        Tissue_eye_Ratio = None  # Handle NaN results
else:
    Tissue_eye_Ratio = None  # Handle division by zero or other issues

# Display the result
print(f"Eye Donations: {donor_eye_count}")
print(f"Tissue Donations: {donor_tissue_count}")
print(f"Ratio of Eye Donations to Tissue Donations: {Tissue_eye_Ratio}")

Eye Donations: 97015
Tissue Donations: 97545
Ratio of Eye Donations to Tissue Donations: 1.0


# 8. How many were cross clamped and Transplanted for each organ type?

In [14]:
# Merging the referral and outcomes sheet
merged_ref_out = pd.merge(referral, outcomes, on="PatientID", how="inner")

# Identify organ columns that start with 'outcome_'
organ_columns = [col for col in merged_ref_out.columns if col.startswith('outcome_')]

# Filter rows where Cross_Clamped = 1 and transplanted = 1
filtered_rows = merged_ref_out[(merged_ref_out['transplanted'] == 1) & (merged_ref_out['Cross_Clamped'] == 1)]

# Initialize a list to store counts for each organ
organ_grouped = []

# Loop through each organ column and count unique PatientID and transplanted for each organ
for organ in organ_columns:
    # Filter for rows where the organ was transplanted (value == 1)
    organ_transplants = filtered_rows[filtered_rows[organ] == 'Transplanted']
    
    # Count unique PatientID values (i.e., the number of transplants for that organ)
    organ_count = organ_transplants['PatientID'].nunique()  # Count unique PatientID
    
    # Append the organ name (without 'outcome_' prefix) and count to the list
    organ_grouped.append([organ.replace('outcome_', ''), organ_count])

# Convert the result into a DataFrame
organ_grouped_df = pd.DataFrame(organ_grouped, columns=['Organ', ' cross clamped and Transplanted Count'])

# Display the grouped results
print("Count of Cross_Clamped and transplanted by organs:")
print(organ_grouped_df)

Count of Cross_Clamped and transplanted by organs:
          Organ   cross clamped and Transplanted Count
0         heart                                   2987
1         liver                                   7014
2   kidney_left                                   7084
3  kidney_right                                   7051
4     lung_left                                   2146
5    lung_right                                   2127
6     intestine                                     85
7      pancreas                                    892


# 9. Are there any donors who are not cross-clamped for organ procurement but transplanted their organs. if so which organ(s)?

In [13]:
# Merging the referral and outcomes sheet
merged_ref_out = pd.merge(referral, outcomes, on="PatientID", how="inner")

# Filter rows where Cross_Clamped = 0 and transplanted = 1
filtered_rows = merged_ref_out[(merged_ref_out['transplanted'] == 1) & (merged_ref_out['Cross_Clamped'] == 0)]

# Display the filtered rows
#print("Rows where Cross_Clamped = 0 and Transplanted = 1:")
#print(filtered_rows)

# Select specific columns to display
selected_columns = filtered_rows[['PatientID', 'Cross_Clamped', 'transplanted', 'outcome_kidney_right']]

# Display the filtered data with selected columns
print("Donor(s) who are not Cross_Clamped but transplanted their organs:")
print(selected_columns)

Donor(s) who are not Cross_Clamped but transplanted their organs:
         PatientID  Cross_Clamped  transplanted outcome_kidney_right
1718  OPO1_P188776              0             1         Transplanted


# 10. Are there any donors who are not Authorized By Family but still transplanted?

In [15]:
# Filter rows where 'Authorized By Family' == 0 and 'transplanted' == 1
referral_rows = referral[(referral['Authorized By Family'] == 0) & (referral['transplanted'] == 1)]

# Select specific columns to display
selected_columns = referral_rows[['PatientID', 'Authorized By Family', 'transplanted']]

# Display the filtered data with selected columns
print("Donor(s) who are not authorized but got Transplanted:")
print(selected_columns)

Donor(s) who are not authorized but got Transplanted:
          PatientID  Authorized By Family  transplanted
20119  OPO1_P188776                     0             1
